In [7]:
import pandas as pd
import numpy as np

In [10]:
data = pd.read_csv('anime.csv')

In [12]:
data.head(10)

,Rus_name,Eng_name,Href,Genres,Description,Studios,Minor_names
0,Стальной алхимик: Братство смотреть онлайн,Hagane no Renkinjutsushi (2009) / FMA (2009) с...,/catalog/hagane-no-renkinjutsushi-2009-2400,"['Экшен', 'Военное', 'Приключения', 'Комедия',...",['Описание анимеРемейк одноимённого аниме-сери...,['/anime/studios/4-bones'],Стальной алхимик: Братство / Hagane no Renkinj...
1,Атака титанов: Финал смотреть онлайн,Shingeki no Kyojin: The Final Season смотреть ...,/catalog/anime-21166,[],[],[],NaN
2,Врата Штейна смотреть онлайн,Steins;Gate смотреть онлайн,/catalog/steins-gate-914,"['Фантастика', 'Триллер']",['Описание анимеДействие сериала происходит ле...,['/anime/studios/314-white-fox'],Врата Штейна / シュタインズ・ゲート / Steins Gate / シュタイ...
3,Атака титанов 3. Часть 2 сезон смотреть онлайн,Shingeki no Kyojin Season 3 Part 2 / SnK3.5 см...,/catalog/anime-19608,[],[],[],NaN
4,Гинтама 4 сезон смотреть онлайн,Gintama° смотреть онлайн,/catalog/gintama-9815,"['Экшен', 'Комедия', 'Исторический', 'Пародия'...",['Описание анимеНикто этого уже не ждал и не н...,['/anime/studios/1258-bandai-namco-pictures'],Gintama° / Гинтама 4 / Gintama (2015) / 銀魂゜ / ...
5,Охотник х Охотник (2011) смотреть онлайн,Hunter x Hunter (2011) смотреть онлайн,/catalog/hunter-x-hunter-2011-4023,"['Экшен', 'Приключения', 'Сёнен', 'Супер сила'...",['Описание анимеРемейк аниме-сериала «Охотник ...,['/anime/studios/11-madhouse-studios'],Hunter x Hunter (2011) / Охотник х Охотник (20...
6,Гинтама 2 сезон смотреть онлайн,Gintama' смотреть онлайн,/catalog/gintama-216,"['Экшен', 'Комедия', 'Исторический', 'Пародия'...","['Описание анимеНад древним Эдо, которым прави...",['/anime/studios/14-sunrise'],Гинтама 2 / Gintama' / Gintama` / 銀魂´ / Gintam...
7,Легенда о героях Галактики смотреть онлайн,Ginga Eiyuu Densetsu смотреть онлайн,/catalog/ginga-eiyuu-densetsu-1016,"['Драма', 'Фантастика', 'Космос', 'Военное']",['Описание анимеДесятилетиями Галактическая Им...,"['/anime/studios/8-artland', '/anime/studios/2...",Ginga Eiyuu Densetsu / Легенда о героях Галакт...
8,Гинтама 3 сезон смотреть онлайн,Gintama': Enchousen смотреть онлайн,/catalog/gintama-enchousen-8813,"['Экшен', 'Комедия', 'Исторический', 'Пародия'...",['Описание анимеТретье пришествие знаменитой т...,['/anime/studios/14-sunrise'],Gintama': Enchousen / Гинтама 3 / Gintama` (20...
9,Форма голоса смотреть онлайн,Koe no Katachi смотреть онлайн,/catalog/koe-no-katachi-9399,"['Драма', 'Школа', 'Сёнен']","['Описание анимеПодобно миру взрослых, мир дет...",['/anime/studios/2-kyoto-animation'],Koe no Katachi / Форма голоса / Eiga Koe no Ka...
